# Basic imports

In [ ]:
import pandas as pd
import os
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams.update({'figure.max_open_warning': 0})

In [ ]:
current_path = os.getcwd()
root_path=current_path.replace('\\forward_feature_selection','')

In [ ]:
mixed_df=pd.read_csv(root_path+"\\molecules.csv",sep="\t")

# Functions

In [ ]:
def save_df_to_disk(df,name:str,separator="\t"):
    df.to_csv(name,sep=separator,index = False, header=True)

In [ ]:
def get_data_and_true_prediction(df,not_wanted_features:list):
        temp_df=df.drop(not_wanted_features,axis=1)
        y=temp_df[temp_df.columns[-1]]
        x=temp_df.drop([temp_df.columns[-1]],axis=1)
        
        return x,y   

In [ ]:
def get_df_with_name_and_prediction(df,true_prediction,big_df):
    new_df=df
    new_df.insert(0,"m_name",big_df["m_name"].values)
    new_df=new_df.join(true_prediction)
    
    return new_df

# Forward selection

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def get_roc_auc_score(x,y,model): # gets roc auc average
        cv_results = cross_validate(model, x, y, cv=10,scoring=('roc_auc'))
        roc_auc_avrg=cv_results['test_score'].mean()
        
        return roc_auc_avrg 

In [ ]:
def KNN_forward_selection(x,y,k:list,pw:list,mt,alfa=6):
    
    def get_best_score_index(score_lst):
        max_score = max(score_lst) # best score
        max_score_index=[i for i, j in enumerate(score_lst) if j == max_score] # indx with best score 
        
        return max_score_index[0]
    
    def inner_iteration(x,y,k,pw:list,mt):
        print("k is now: {}".format(k))
        df_score_lst = []
        for i in pw:
            KNB_clf = KNeighborsClassifier(n_neighbors=k,p=i,metric=mt) # KNN model
            df,score = forward_selection(x,y,KNB_clf)
            df_score_lst.append([df,score])
            
        score_lst = [y for [x,y] in df_score_lst]
        best_index = get_best_score_index(score_lst)
        print("Best inner model when p = {}".format(pw[best_index]))
        print("\nFeatures\n")
        for c,i in enumerate(df_score_lst[best_index][0].columns):
            print("{}. {}".format(c+1,i))
        
        return df_score_lst[best_index][0],df_score_lst[best_index][1],pw[best_index]
        
    print(f"Metric: {mt}\n")
    
    outer_df_score_lst = []
    temp_best_score = 0
    
    tries = math.ceil((len(k)/alfa))+1 # spare tries if by some point the performance decreases
    
    for cnt,i in enumerate(k):
        best_inner_df,best_inner_score,best_p_value = inner_iteration(x,y,i,pw,mt)
        print("\nSpare tries: {}".format(tries))
        
        if((best_inner_score > temp_best_score) and tries > 0):
            temp_best_score = best_inner_score
            outer_df_score_lst.append([best_inner_df,best_inner_score,i,best_p_value])
            if (cnt > 0): print("This iteration had an improvement\n")
            elif (cnt == 0): print("")             
            tries = math.ceil((len(k)/alfa))+1            
        else:
            print("This iteration didn't have an improvement\n")
            tries = tries-1
        
        if (tries == 0):
            break
                                              
    outer_score_lst = [b for [a,b,c,d] in outer_df_score_lst]
    best_outer_index = get_best_score_index(outer_score_lst)
    
    print("Final results")
    print("Best model when k = {} ,p = {} ,roc_auc = {}%".format(outer_df_score_lst[best_outer_index][2],
                                                                 outer_df_score_lst[best_outer_index][3],
                                                                 outer_df_score_lst[best_outer_index][1]*100))
    
    return outer_df_score_lst[best_outer_index][0],outer_df_score_lst[best_outer_index][1]   

# Data

In [ ]:
unnecesary_features=["m_name"]
x,y=get_data_and_true_prediction(mixed_df,unnecesary_features)

# Comparison KNN

In [ ]:
max_k = 50
max_p = 15

k_lst = [i for i in range(1,max_k+1)]
p_lst = [i for i in range(1,max_p+1)]

p = 1, Manhattan Distance
p = 2, Euclidean Distance
p = ∞, Chebychev Distance

In [ ]:
start = time.time()
df,score_normal=KNN_forward_selection(x,y,k_lst,p_lst,'minkowski')
end = time.time()

time.strftime('%H:%M:%S', time.gmtime(end-start))